In [1]:
import mysql.connector
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
from sqlalchemy import create_engine, text
load_dotenv(find_dotenv())

True

In [2]:
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

In [3]:
user

'xsylitx3w6v21qdtinlf'

In [4]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
connection = engine.connect()
result = connection.execute(text("SHOW TABLES;"))
for r in result.mappings():
    print(r)
connection.close()



2024-02-11 06:36:15,213 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 06:36:15,214 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:15,393 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 06:36:15,393 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:15,487 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 06:36:15,487 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:15,625 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 06:36:15,626 INFO sqlalchemy.engine.Engine SHOW TABLES;
2024-02-11 06:36:15,627 INFO sqlalchemy.engine.Engine [generated in 0.03181s] {}
{'Tables_in_ehrudite-alt': 'ADMISSIONS'}
{'Tables_in_ehrudite-alt': 'CHARTEVENTS'}
{'Tables_in_ehrudite-alt': 'COST'}
{'Tables_in_ehrudite-alt': 'DIAGNOSES_ICD'}
{'Tables_in_ehrudite-alt': 'D_ICD_DIAGNOSES'}
{'Tables_in_ehrudite-alt': 'D_ICD_PROCEDURES'}
{'Tables_in_ehrudite-alt': 'D_ITEMS'}
{'Tables_in_ehrudite-alt': 'D_LABITEMS'}
{'Tables_in_ehru

In [5]:
f = open("mimic_iii.sql")
lines = [l + ';' for l in "".join(f.readlines()).replace("\n", "").split(";")][:-1]
for l in lines: print(l)

DROP TABLE IF EXISTS PATIENTS;
CREATE TABLE PATIENTS (    ROW_ID INT NOT NULL PRIMARY KEY,    SUBJECT_ID INT NOT NULL UNIQUE,    GENDER VARCHAR(5) NOT NULL,    DOB DATETIME(0) NOT NULL,    DOD DATETIME(0));
DROP TABLE IF EXISTS ADMISSIONS;
CREATE TABLE ADMISSIONS(    ROW_ID INT NOT NULL PRIMARY KEY,    SUBJECT_ID INT NOT NULL,    HADM_ID INT NOT NULL UNIQUE,    ADMITTIME DATETIME(0) NOT NULL,    DISCHTIME DATETIME(0),    ADMISSION_TYPE VARCHAR(50) NOT NULL,    ADMISSION_LOCATION VARCHAR(50) NOT NULL,    DISCHARGE_LOCATION VARCHAR(50),    INSURANCE VARCHAR(255) NOT NULL,    LANGUAGE VARCHAR(10),    MARITAL_STATUS VARCHAR(50),    ETHNICITY VARCHAR(200) NOT NULL,    AGE INT NOT NULL,    FOREIGN KEY(SUBJECT_ID) REFERENCES PATIENTS(SUBJECT_ID));
DROP TABLE IF EXISTS D_ICD_DIAGNOSES;
CREATE TABLE D_ICD_DIAGNOSES(    ROW_ID INT NOT NULL PRIMARY KEY,    ICD9_CODE VARCHAR(10) NOT NULL UNIQUE,    SHORT_TITLE VARCHAR(50) NOT NULL,    LONG_TITLE VARCHAR(255) NOT NULL);
DROP TABLE IF EXISTS D_ICD_P

In [6]:
cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database)

cursor = cnx.cursor()
# cursor.execute(f)
# cursor.execute(f)
for l in lines:
    if l[:2] == '--': continue
    print(l)
    cursor.execute(l)
version = cursor.fetchone()
display(version, cursor)

cnx.close()


DROP TABLE IF EXISTS PATIENTS;


DatabaseError: 3730 (HY000): target: ehrudite-alt.-.primary: vttablet: rpc error: code = Unknown desc = Cannot drop table 'PATIENTS' referenced by a foreign key constraint 'ADMISSIONS_ibfk_1' on table 'ADMISSIONS'. (errno 3730) (sqlstate HY000) (CallerID: xsylitx3w6v21qdtinlf): Sql: "drop table if exists PATIENTS", BindVars: {REDACTED}

In [ ]:
f = open("mimic-iii-teardown.sql")
t_lines = [l + ';' for l in "".join(f.readlines()).replace("\n", "").split(";")][:-1]
for l in t_lines: print(l)

DELETE FROM DIAGNOSES_ICD;
DELETE FROM LABEVENTS;
DELETE FROM PRESCRIPTIONS;
DELETE FROM PROCEDURES_ICD;
DELETE FROM ADMISSIONS;
DELETE FROM PATIENTS;
DELETE FROM D_LABITEMS;
DELETE FROM D_ITEMS;
DELETE FROM D_ICD_DIAGNOSES;


In [7]:
cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
    database=database)

cursor = cnx.cursor()
# cursor.execute(f)
# cursor.execute(f)
for l in t_lines:
    if l[:2] == '--': continue
    print(l)
    cursor.execute(l)
version = cursor.fetchone()
display(version, cursor)

cnx.close()


NameError: name 't_lines' is not defined

In [8]:
def read_csv(data_dir, filename, columns=None, lower=True, filter=None, dtype=None, memory_efficient=False):
    filepath = os.path.join(data_dir, filename)
    if memory_efficient:
        import dask.dataframe as dd
        from dask.diagnostics import ProgressBar
        ProgressBar().register()

        if filepath.endswith('gz'):
            compression='gzip'
        else:
            compression=None

        if dtype:
            df = dd.read_csv(filepath, blocksize=25e6, dtype=dtype, compression=compression)
        else:
            df = dd.read_csv(filepath, blocksize=25e6, compression=compression)
        if columns is not None:
            df = df[columns]
        if filter is not None:
            for key in filter:
                df = df[df[key].isin(filter[key])]
        df = df.compute()
    else:
        df = pd.read_csv(filepath, usecols=columns)
        if filter is not None:
            for key in filter:
                df = df[df[key].isin(filter[key])]
    if lower:
        df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)
    return df

In [9]:
conn_string = f'mysql+mysqlconnector://{user}:{password}@{host}/{database}'
engine = create_engine(conn_string, echo=True)
conn = engine.connect()

import math

def to_sqlW(df, title, conn, if_exists=None, index=None):
    print(f"{title}: {len(df)}")
    # frac = min(1, 60000 / len(df))
    # display(frac)

    if len(df) > 30000:
        for i in range(math.ceil(len(df) / 30000)):
        # for i in range(0, 2):
            df1 = df[i * 30000 : min(len(df), (i + 1) * 30000)]
            display(i * 30000, min(len(df), (i + 1) * 30000))
            df1.to_sql(title, conn, if_exists=if_exists, index=index)
        
        return
        
    display(df1)
    # try:
    df1.to_sql(title, conn, if_exists=if_exists, index=index)
    # except:
        # print("failed")
    # print(df.size)

# rows = read_csv("./", "PATIENTS.csv")
# to_sqlW(rows, "PATIENTS", conn, if_exists="append", index=False)

# rows = read_csv("./", "ADMISSIONS.csv")
# to_sqlW(rows, "ADMISSIONS", conn, if_exists="append", index=False)

# rows = read_csv("./", "D_ICD_DIAGNOSES.csv").astype({"icd9_code": str})
# to_sqlW(rows, "D_ICD_DIAGNOSES", conn, if_exists="append", index=False)

# rows = read_csv("./", "D_ICD_PROCEDURES.csv").astype({"icd9_code": str})
# to_sqlW(rows, "D_ICD_PROCEDURES", conn, if_exists="append", index=False)

# rows = read_csv("./", "D_ITEMS.csv")
# to_sqlW(rows, "D_ITEMS", conn, if_exists="append", index=False)

# rows = read_csv("./", "D_LABITEMS.csv")
# to_sqlW(rows, "D_LABITEMS", conn, if_exists="append", index=False)

# rows = read_csv("./", "DIAGNOSES_ICD.csv").astype({"icd9_code": str})
# to_sqlW(rows, "DIAGNOSES_ICD", conn, if_exists="append", index=False)

# rows = read_csv("./", "PROCEDURES_ICD.csv").astype({"icd9_code": str})
# to_sqlW(rows, "PROCEDURES_ICD", conn, if_exists="append", index=False)

# rows = read_csv("./", "LABEVENTS.csv")
# to_sqlW(rows, "LABEVENTS", conn, if_exists="append", index=False)

# rows = read_csv("./", "PRESCRIPTIONS.csv")
# to_sqlW(rows, "PRESCRIPTIONS", conn, if_exists="append", index=False)

# rows = read_csv("./", "ICUSTAYS.csv")
# to_sqlW(rows, "ICUSTAYS", conn, if_exists="append", index=False)

# rows = read_csv("./", "INPUTEVENTS_CV.csv")
# to_sqlW(rows, "INPUTEVENTS_CV", conn, if_exists="append", index=False)

# rows = read_csv("./", "OUTPUTEVENTS.csv")
# to_sqlW(rows, "OUTPUTEVENTS", conn, if_exists="append", index=False)

# rows = read_csv("./", "MICROBIOLOGYEVENTS.csv")
# to_sqlW(rows, "MICROBIOLOGYEVENTS", conn, if_exists="append", index=False)

# rows = read_csv("./", "TRANSFERS.csv")
# to_sqlW(rows, "TRANSFERS", conn, if_exists="append", index=False)

# rows = read_csv("./", "COST.csv")
# df1 = read_csv("./", "COST.csv")

# df2 = read_csv("./", "DIAGNOSES_ICD.csv").astype({"icd9_code": str})
# # display(df2)

# rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['icd9_code'])
# rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
# rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "icd9_code", "charttime"])
# # display(rows)

# df2 = read_csv("./", "LABEVENTS.csv")
# # display(df2)

# rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['valueuom'])
# rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
# rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "charttime", "itemid", "valuenum", "valueuom"])
# # display(rows)

# df2 = read_csv("./", "PROCEDURES_ICD.csv")
# # display(df2)
# # print("hi")

# rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['icd9_code'])
# rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
# rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "icd9_code", "charttime",])
# display(rows)

# to_sqlW(rows, "COST", conn, if_exists="append", index=False)

rows = read_csv("./", "CHARTEVENTS.csv")
to_sqlW(rows, "CHARTEVENTS", conn, if_exists="append", index=False)

conn.close()

2024-02-11 06:36:31,568 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-02-11 06:36:31,569 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:31,712 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-02-11 06:36:31,713 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:31,784 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-02-11 06:36:31,784 INFO sqlalchemy.engine.Engine [raw sql] {}


/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_42976/2375272697.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)


CHARTEVENTS: 227180


0

30000

2024-02-11 06:36:32,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-11 06:36:32,859 INFO sqlalchemy.engine.Engine DESCRIBE `ehrudite-alt`.`CHARTEVENTS`
2024-02-11 06:36:32,859 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-11 06:36:33,094 INFO sqlalchemy.engine.Engine INSERT INTO `CHARTEVENTS` (row_id, subject_id, hadm_id, icustay_id, itemid, charttime, valuenum, valueuom) VALUES (%(row_id)s, %(subject_id)s, %(hadm_id)s, %(icustay_id)s, %(itemid)s, %(charttime)s, %(valuenum)s, %(valueuom)s)
2024-02-11 06:36:33,094 INFO sqlalchemy.engine.Engine [generated in 0.11612s] [{'row_id': 0, 'subject_id': 40124, 'hadm_id': 126179, 'icustay_id': 279554.0, 'itemid': 646, 'charttime': '2023-03-02 04:00:00', 'valuenum': 96.0, 'valueuom': '%'}, {'row_id': 1, 'subject_id': 40124, 'hadm_id': 126179, 'icustay_id': 279554.0, 'itemid': 733, 'charttime': '2023-03-02 04:25:00', 'valuenum': 0.0, 'valueuom': 'kg'}, {'row_id': 2, 'subject_id': 40124, 'hadm_id': 126179, 'icustay_id': 279554.0, 'i

IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): target: ehrudite-alt.-.primary: vttablet: rpc error: code = AlreadyExists desc = Duplicate entry '0' for key 'CHARTEVENTS.PRIMARY' (errno 1062) (sqlstate 23000) (CallerID: xsylitx3w6v21qdtinlf): Sql: "insert into CHARTEVENTS(row_id, subject_id, hadm_id, icustay_id, itemid, charttime, valuenum, valueuom) values (:vtg1 /* INT64 */, :vtg2 /* INT64 */, :vtg3 /* INT64 */, :vtg4 /* DECIMAL */, :vtg5 /* INT64 */, :vtg6 /* VARCHAR */, :vtg7 /* DECIMAL */, :vtg8 /* VARCHAR */), (:vtg9 /* INT64 */, :vtg10 /* INT64 *
[SQL: INSERT INTO `CHARTEVENTS` (row_id, subject_id, hadm_id, icustay_id, itemid, charttime, valuenum, valueuom) VALUES (%(row_id)s, %(subject_id)s, %(hadm_id)s, %(icustay_id)s, %(itemid)s, %(charttime)s, %(valuenum)s, %(valueuom)s)]
[parameters: [{'row_id': 0, 'subject_id': 40124, 'hadm_id': 126179, 'icustay_id': 279554.0, 'itemid': 646, 'charttime': '2023-03-02 04:00:00', 'valuenum': 96.0, 'valueuom': '%'}, {'row

In [71]:

rows = read_csv("./", "COST.csv")
# df1 = read_csv("./", "COST.csv")

df2 = read_csv("./", "DIAGNOSES_ICD.csv").astype({"icd9_code": str})
# display(df2)

rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['icd9_code'])
rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "icd9_code", "charttime"])
# display(rows)

df2 = read_csv("./", "LABEVENTS.csv")
# display(df2)

rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['valueuom'])
rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "charttime", "itemid", "valuenum", "valueuom"])
# display(rows)

df2 = read_csv("./", "PROCEDURES_ICD.csv")
# display(df2)
# print("hi")

rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['icd9_code'])
rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "icd9_code", "charttime",])
# display(rows)

# df2 = read_csv("./", "LABEVENTS.csv")
# # display(df2)

# rows = pd.merge(rows, df2, left_on='event_id', right_on='row_id', how='left').dropna(subset=['valueuom'])
# rows.rename(columns={"row_id_x": "row_id", "subject_id_x": "subject_id", "hadm_id_x": "hadm_id"}, inplace=True)
# rows = rows.drop(columns=["row_id_y", "subject_id_y", "hadm_id_y", "charttime", "itemid", "valuenum", "valueuom"])
# display(rows)

/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_99962/2375272697.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)
/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_99962/2375272697.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)


,row_id,subject_id,hadm_id,event_type,event_id,chargetime,cost
0,0,10006,142345.0,diagnoses_icd,0,2022-11-27 21:09:00,13.83
1,1,10006,142345.0,diagnoses_icd,1,2022-11-27 21:09:00,13.29
2,2,10006,142345.0,diagnoses_icd,2,2022-11-27 21:09:00,13.32
3,3,10006,142345.0,diagnoses_icd,3,2022-11-27 21:09:00,15.19
4,4,10006,142345.0,diagnoses_icd,4,2022-11-27 21:09:00,14.98
...,...,...,...,...,...,...,...
51996,51996,10117,105150.0,prescriptions,1529,2022-12-07 00:00:00,13.69
51997,51997,10117,105150.0,prescriptions,1530,2022-12-07 00:00:00,14.74
51998,51998,10117,105150.0,prescriptions,1531,2022-12-08 00:00:00,12.74
51999,51999,10117,105150.0,prescriptions,1532,2022-12-10 00:00:00,13.76


/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_99962/2375272697.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)


,row_id,subject_id,hadm_id,itemid,charttime,valuenum,valueuom
0,0,10006,142345.0,50893,2022-11-27 17:33:00,7.80,mg/dl
1,1,10006,142345.0,50912,2022-11-27 17:38:00,3.30,mg/dl
2,2,10006,142345.0,51003,2022-11-27 17:38:00,0.02,ng/ml
3,3,10006,142345.0,51006,2022-11-27 17:38:00,12.00,mg/dl
4,4,10006,142345.0,51250,2022-11-27 17:38:00,91.00,fl
...,...,...,...,...,...,...,...
48455,48455,44222,192189.0,51200,2023-08-28 04:00:00,1.70,%
48456,48456,44222,192189.0,51250,2023-08-28 04:00:00,74.00,fl
48457,48457,44222,192189.0,50878,2023-08-28 04:00:00,21.00,iu/l
48458,48458,44222,192189.0,50885,2023-08-28 04:00:00,0.30,mg/dl


/var/folders/kh/by6j2m994_d3p3lgsnb74fw00000gp/T/ipykernel_99962/2375272697.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower().strip() if pd.notnull(x) and type(x)==str else x)


In [ ]:
t = "select distinct cost.cost from cost where cost.event_type = 'procedures_icd' and cost.event_id in ( select procedures_icd.row_id from procedures_icd where procedures_icd.icd9_code = ( select d_icd_procedures.icd9_code from d_icd_procedures where d_icd_procedures.short_title = 'inject/infuse nec"
t.upper()